In [ ]:
import SimpleITK as sitk
import radiomics
from radiomics import firstorder, glcm, ngtdm, shape, imageoperations, getFeatureClasses
import os, glob, shutil, tempfile
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchsummary as ts
import monai
import logging
import nibabel as nib
import six

from monai.config import print_config
from monai.data import (
    ArrayDataset,
    GridPatchDataset,
    PatchIter,
    PILReader
    )
from monai.transforms import (
    Compose,
    LoadImage,
    LoadImaged,
    RandSpatialCrop,
    ScaleIntensity,
    EnsureType,
)
from monai.utils import first
print_config()

In [11]:
path = "./Data/"
temp_list = os.listdir(path)
cmb_list = []
swi_list = []
seg_list = []
for i, name in enumerate(temp_list):
    if "." in  name:
        temp_list.pop(i)
    else:
        cmb_list.append(sitk.ReadImage(path+name+"/cmb.nii.gz"))
        swi_list.append(sitk.ReadImage(path+name+"/swi.nii.gz"))
        seg_list.append(sitk.ReadImage(path+name+"/lm-swi.nii.gz"))

In [6]:
# Loggers
logger = radiomics.logger
logger.setLevel(logging.DEBUG)
handler = logging.FileHandler(filename="testLog.txt", mode="w")
formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

In [67]:
# Radiomics extractor setting
lb1_settings = {
    "binWidth":10,
    "minimumROIDimensions":2,
    "minumumROISize":None,
    "normalize":True, # weighted images = True
    "normalizeScale":100,
    "removeOutliers":None,
    "resampledPixelSpacing":None, # for sampling
    "interpolator":None,
    "preCrop":True,
    "padDistance":1,
    "label":0,
    "additionalInfo":True,
    "voxelArrayShift":300
}
lb2_settings = {
   
}
lb3_settings = {
    "binWidth":10,
    "minimumROIDimensions":2,
    "minumumROISize":None,
    "normalize":True, # weighted images = True
    "normalizeScale":100,
    "removeOutliers":None,
    "resampledPixelSpacing":None, # for sampling
    "interpolator":None,
    "preCrop":True,
    "padDistance":1,
    "label":2,
    "additionalInfo":True,
    "voxelArrayShift":300
}

In [40]:
def rfe_commence(orig, mask, settings):
    fo_features = firstorder.RadiomicsFirstOrder(orig, mask, **settings)
    fo_features.disableAllFeatures()
    fo_features.enableAllFeatures()
    sh_features = shape.RadiomicsShape(orig, mask, **settings)
    sh_features.disableAllFeatures()
    sh_features.enableAllFeatures()
    glcm_features = glcm.RadiomicsGLCM(orig, mask, **settings)
    glcm_features.disableAllFeatures()
    glcm_features.enableAllFeatures()
    ngtdm_features = ngtdm.RadiomicsNGTDM(orig, mask, **settings)
    ngtdm_features.disableAllFeatures()
    ngtdm_features.enableAllFeatures()
    
    fo_result = fo_features.execute()
    sh_result = sh_features.execute()
    glcm_result = glcm_features.execute()
    ngtdm_result = ngtdm_features.execute()
    
    def zscore(data):# I'm not sure in which scope i should apply this
        mean = np.mean(data)
        std = np.std(data)
        z_scores = [(x - mean) / std for x in data]
        return z_scores

    def packup(fo, sh, glcm, ngtdm):# hit?
        fo = [val for val in six.itervalues(fo)]
        fo_score = zscore(fo)
        sh = [val for val in six.itervalues(sh)]
        sh_score = zscore(sh)
        glcm = [val for val in six.itervalues(glcm)]
        glcm_score = zscore(glcm)
        ngtdm = [val for val in six.itervalues(ngtdm)]
        ngtdm_score = zscore(ngtdm)
        return {"fo":fo_score, "sh":sh_score, "glcm":glcm_score, "ngtdm":ngtdm_score}
    
    result_dic = packup(fo_result, sh_result, glcm_result, ngtdm_result)
    return result_dic
    

In [ ]:
target = 2
benign_result = rfe_commence(swi_list[target], cmb_list[target], lb1_settings)
cmb_result = rfe_commence(swi_list[target], cmb_list[target], lb2_settings) 
cal_result = rfe_commence(swi_list[target], cmb_list[target], lb3_settings)

In [68]:
cmb_result = rfe_commence(swi_list[target], cmb_list[target], lb2_settings)

2023-10-25 01:44:32,098 - Initializing feature class
2023-10-25 01:44:32,149 - Discretizing gray levels inside ROI


ValueError: min() arg is an empty sequence

In [20]:
len(cmb_list[2])

9011200

In [70]:
torch.cuda.is_available()

False

In [45]:
print(f'first order: {len(result["fo"])}\nshape: {len(result["sh"])}\nglcm: {len(result["glcm"])}\nngtdm: {len(result["ngtdm"])}')

first order: 18
shape: 14
glcm: 24
ngtdm: 5


<hr style="color:lightblue">
INPUT
<hr style="color:lightblue">

In [70]:
swi_monai = LoadImage(image_only=True, ensure_channel_first=True, simple_keys=True)(swi_list[0])
cmb_monai = LoadImage(image_only=True, ensure_channel_first=True, simple_keys=True)(cmb_list[0])
t1_monai = LoadImage(image_only=True, ensure_channel_first=True, simple_keys=True)(t1_list[0])
# it's a tensor

In [71]:
swi_monai.shape

torch.Size([1, 320, 352, 80])

In [74]:
input_meta = torch.tensor(result["fo"]+result["sh"]+result["glcm"]+result["ngtdm"])

In [76]:
input_meta = input_meta.view(1, -1) # add one channel
input_meta.shape

<hr style="color:pink">
TENSOR CONCAT TEST
<hr style="color:pink">

In [135]:
concat_test = torch.cat((torch.tensor([[-1, -3, -5, -7]]), torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 19]])), dim= 1)
print(concat_test)
print(concat_test.shape)

tensor([[-1, -3, -5, -7,  1,  2,  3,  4,  5,  6,  7,  8,  9, 19]])
torch.Size([1, 14])


<hr style="color:lightgreen">
LAYERS
<hr style="color:lightgreen">

In [ ]:
classNum = 3
class CMBWATCHDOG(nn.Module):
    def __init__(self, classes):
        super(CMBWATCHDOG, self).__init__()
        
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3)
        self.maxp1 = nn.MaxPool3d(kernel_size=2)
        
        self.bnorm = nn.BatchNorm3d()
        
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3)
        self.maxp2 = nn.MaxPool3d(kernel_size=2)
        
        self.flat = nn.Flatten()
        
        self.dense1 = nn.Linear(64, 128)
        self.drop = nn.Dropout(0.5)
        
        self.dense2 = nn.Linear(128, 61)
        
        self.concat = 
        
    def forward(self, x):
        
        x = torch.relu(self.conv1(x))
        x = self.maxp1(x)
        x = self.bnorm(x)
        x = torch.relu(self.conv2(x))
        x = self.maxp2(x)
        x = self.bnorm(x)
        
        x = self.flat(x)
        
        x = self.dense1(x)
        x = self.drop(x)
        x = self.dense2(x)
        x = self.drop(x)
        return x
    
model = CMBWATCHDOG(classNum)
ts.summary(model, (2,2,2))